# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [3]:
install.packages("boot")
install.packages("mlbench")
install.packages("tidyverse")
install.packages("ggplot2")
install.packages("vctrs", dependencies = TRUE)
install.packages("dplyr")
library(tidyverse)
library(boot)
library(mlbench)
library(ggplot2)
library(dplyr)
data(PimaIndiansDiabetes2)
data <- PimaIndiansDiabetes2
data <- data %>% select(-c(insulin))
data <- na.omit(data)
data_new <- data

head(data_new)
dim(data_new)
str(data_new)



Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘downlit’, ‘pkgdown’


Warning message in install.packages("vctrs", dependencies = TRUE):
“installation of package ‘downlit’ had non-zero exit status”
Warning message in install.packages("vctrs", dependencies = TRUE):
“installation of package ‘pkgdown’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


[1] 532   8

'data.frame':	532 obs. of  8 variables:
 $ pregnant: num  6 1 1 0 3 2 1 5 0 1 ...
 $ glucose : num  148 85 89 137 78 197 189 166 118 103 ...
 $ pressure: num  72 66 66 40 50 70 60 72 84 30 ...
 $ triceps : num  35 29 23 35 32 45 23 19 47 38 ...
 $ mass    : num  33.6 26.6 28.1 43.1 31 30.5 30.1 25.8 45.8 43.3 ...
 $ pedigree: num  0.627 0.351 0.167 2.288 0.248 ...
 $ age     : num  50 31 21 33 26 53 59 51 31 33 ...
 $ diabetes: Factor w/ 2 levels "neg","pos": 2 1 1 2 2 2 2 2 2 1 ...
 - attr(*, "na.action")= 'omit' Named int [1:236] 3 6 8 10 11 12 13 16 18 22 ...
  ..- attr(*, "names")= chr [1:236] "3" "6" "8" "10" ...


(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [19]:
#index for first prediction 
holdout_index <- 1 

#logistic regression model on all observations except the holdout index
logistic_model <- glm(diabetes ~ ., data = data_new[-holdout_index, ], family = binomial(link = "logit"))

#predict log odds probability of diabetes for the holdout index
predicted_prob <- predict(logistic_model, newdata = data_new[holdout_index, ], type = "link")

#convert the predicted log odds to probability
predicted_prob <- exp(predicted_prob) / (1 + exp(predicted_prob))

#display the predicted probability
predicted_prob

#get the actual class of diabetes for the holdout index
actual_class <- data_new[holdout_index, "diabetes"]

#check if the predicted class matches the actual class
predicted_class <- ifelse(predicted_prob > 0.5, "pos", "neg")
predicted_class

#print results
print(paste("Predicted probability of diabetes for holdout index", holdout_index, ":", predicted_prob)) 
print(paste("Actual class of diabetes for holdout index", holdout_index, ":", actual_class))



1 
0.729487

1 
"pos"

[1] "Predicted probability of diabetes for holdout index 1 : 0.729486981746287"
[1] "Actual class of diabetes for holdout index 1 : pos"


So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [9]:
# Initialize `results` data frame
n <- nrow(data_new)
results <- data.frame(index = 1:n, actual = data_new$diabetes, predicted = NA)

#for loop
for (i in 1:nrow(data_new)){ #don't forget to change this to your data set name
    # separate individual observation `i` from the rest of your data
    train_data <- data_new[-i, ]
    test_data <- data_new[i, ]

    # train your model
    model <- glm(diabetes ~ ., data = data_new[-i, ], family = binomial(link = "logit"))

    # test model on hold out observation
    prediction_log_odds <- predict(model, newdata = data_new[i, ], type = "link")

    # classify model prediction as "pos" or "neg" and add to `results`
    results$predicted[i] <- ifelse(prediction_log_odds > 0.0, "pos", "neg")

}

# print first few results
head(results)

,index,actual,predicted
,<int>,<fct>,<chr>
1,1,pos,pos
2,2,neg,neg
3,3,neg,neg
4,4,pos,pos
5,5,pos,neg
6,6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [12]:
# After the loop, calculate the accuracy
accuracy <- mean(results$actual == results$predicted)

# Calculate the error rate
error_rate <- 1 - accuracy

# print results
print(paste("Error rate of the model:", round(1 - accuracy, 4)))
print(paste("Accuracy of the model:", round(1 - error_rate, 4)))



[1] "Error rate of the model: 0.2218"
[1] "Accuracy of the model: 0.7782"


# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [ ]:
#?cv.glm

Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [24]:
# reload the library
library(boot)

# define the cost function
#this function calculates the misclassification error
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)

# fit the logistic regression model
logistic_model <- glm(diabetes ~ ., data = data_new, family = binomial(link = "logit"))

# perform cross-validation
cv_results <- cv.glm(data_new, logistic_model, cost)

# print the cross-validation error
cv_error_rate <- cv_results$delta[1]
print(paste("Cross-validation error from cv.glm:", round(cv_error_rate, 4)))
print(paste("Error rate from previous calculation", round(error_rate, 4)))


[1] "Cross-validation error from cv.glm: 0.2218"
[1] "Error rate from previous calculation 0.2218"


How do your results compare to your manual LOOCV above?

> * *They match exactly.*


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [29]:
set.seed(1)
#INSERT CODE BELOW

# reload the library
library(boot)

# define the cost function
#this function calculates the misclassification error
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)

# fit the logistic regression model
logistic_model <- glm(diabetes ~ ., data = data_new, family = binomial(link = "logit"))

# K = 3
cv_results_k3 <- cv.glm(data_new, logistic_model, cost, K = 3)

# K = 5
cv_results_k5 <- cv.glm(data_new, logistic_model, cost, K = 5)

# K = 10
cv_results_k10 <- cv.glm(data_new, logistic_model, cost, K = 10)

# K = 15
cv_results_k15 <- cv.glm(data_new, logistic_model, cost, K = 15)

# results
print(paste("Cross-validation error from cv.glm with K = 3:", round(cv_results_k3$delta[1], 4)))
print(paste("Cross-validation error from cv.glm with K = 5:", round(cv_results_k5$delta[1], 4)))
print(paste("Cross-validation error from cv.glm with K = 10:", round(cv_results_k10$delta[1], 4)))
print(paste("Cross-validation error from cv.glm with K = 15:", round(cv_results_k15$delta[1], 4)))

[1] "Cross-validation error from cv.glm with K = 3: 0.2105"
[1] "Cross-validation error from cv.glm with K = 5: 0.2162"
[1] "Cross-validation error from cv.glm with K = 10: 0.2237"
[1] "Cross-validation error from cv.glm with K = 15: 0.2274"


#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> * *As K increases, the error rate also increases, with k = 3 and k = 5 having lower rates than the LOOCV error and k = 10 and k = 15 higher than the LOOCV.*


If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
>  *I wouldn't expect different results b/c the default k is k=n so each entry is its own test and therefore used only once.*



**DUE:** 5pm March 25, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *my coding tutor, claude.ai*
>
>
